In [1]:
import numpy as np
from numpy import linalg as LA
import tensorflow as tf
import tensornetwork
from tensornetwork import ncon
net = tensornetwork.TensorNetwork()
sess = tf.InteractiveSession()

# Centro de ortogonalidad (centrado en enlace)

Fijamos un indice interno (o enlace) dentro de la red como un centro de ortogonalidad.


1.   Introducimos una matriz de enlace $\sigma$ situada en el enlace en cuestion que va a ser definida inicialmente como la matriz identidad (no cambia las propiedades de la red).
2.   Luego, usamos algunos de los metodos vistos para fijar $\sigma$ como centro de ortogonalidad.



<br>
<b>Ejemplo:</b> Uso del metodo de ortogonalizacion directa en el enlace $B$-$C$ para establecerlo como centro de ortogonalidad de la red $\{A,B,C\}$

<img src="https://static.wixstatic.com/media/d91e93_1d0acbb1faea468daf2a5da7e96ab211~mv2.png/v1/fill/w_900,h_554,al_c,q_90/fig3.webp" width="500">

In [4]:
##### Establecer enlace B-C como centro de ortogonalidad
d = 5 # dimension de indices
A = np.random.rand(d,d,d)
B = np.random.rand(d,d,d)
C = np.random.rand(d,d,d)
Sig = np.eye(d); # matriz inicial (identidad) del enlace

# generar una transformacion de gauge de las matrices
rho1 = ncon([A,A.T,B,B.T],[[1,2,3],[1,2,4],[3,5,-1],[4,5,-2]])
rho2 = ncon([C,C.T],[[-1,1,2],[-2,1,2]])
d1, u1 = LA.eigh(rho1.eval()); sq_d1 = np.sqrt(abs(d1))
d2, u2 = LA.eigh(rho2.eval()); sq_d2 = np.sqrt(abs(d2))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T
# implementar transformacion de gauge
Bprime = ncon([B,X1inv],[[-1,-2,1],[1,-3]])
Cprime = ncon([X2inv,C],[[-1,1],[1,-2,-3]])
Sig_prime = X1 @ Sig @ X2
# verificar
H0 = ncon([A,B,C],[[-1,-2,1],[1,-3,2],[2,-4,-5]])
H1 = ncon([A,Bprime,Sig_prime,Cprime],[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]])
print("totErr = ",LA.norm(H0.eval() - H1.eval()) / LA.norm(H0.eval()))

totErr =  1.197686305236307e-15


Ligadura adicional: 

*  el gauge debe ser elegido de forma que al final la matriz de enlace sea diagonal con elementos positivos en orden descendente de magnitud. 
*  Se necesita un paso final en donde se hace la descomposicion SVD de $\sigma'$ y luego contraemos las matrices $U$ y $V^†$ con los tensores conectados con $\sigma'$ para dejarla como matriz diagonal.

<img src="https://static.wixstatic.com/media/d91e93_301718cb104b441c9e6528490cbc2c7e~mv2.png/v1/fill/w_825,h_413,al_c,q_90/fig4.webp" width="500">

In [5]:
##### Establecer un enlace como centro de ortogonalidad
##### diagonalizacion de matriz de enlace

# Realizar cambio gauge unitario para diagonalizar la matriz de enlace
utemp, Sig_pp, vhtemp = LA.svd(Sig_prime,full_matrices=False)
Bpp = ncon([Bprime,utemp],[[-1,-2,1],[1,-3]])
Cpp = ncon([Cprime,vhtemp],[[1,-2,-3],[-1,1]])
# verificar
H2 = ncon([A,Bpp,np.diag(Sig_pp),Cpp],[[-1,-2,1],[1,-3,2],[2,3],[3,-4,-5]])
print("totErr = ",LA.norm(H0.eval() - H2.eval()) / LA.norm(H0.eval()))

totErr =  1.2466401138274717e-15


## Equivalencia del centro de ortogonalidad de enlace y el SVD

El resultado de fijar un enlace dentro de una red como centro de ortogonalidad es equivalente a un SVD: 

*  La matriz de enlace resultante $\sigma^{''}$ contiene los valores singulares 
*   $U_H$ = ($A⋅ B^{''}$) y ($V_H$)$^†$ = $C^{''}$, hasta para la posibilidad de signos positivos/negativos

<img src="https://static.wixstatic.com/media/d91e93_994b998eb81448248c6a343c0b2b874e~mv2.png/v1/fill/w_888,h_435,al_c,q_90/fig6.webp" width="500">

**Observacion:**

*  Suponiendo que los valores singulares no son degenerados, se sabe que la SVD es única hasta para los signos de los vectores singulares (o fases de los vectores singulares para matrices complejas).

*  Un enlace como centro de ortogonalidad puede fijar completamente la libertad de calibracion de este enlace (En general, esto es verdadero si los elementos de la matriz de enlace no son degenerados).  Esta propiedad de unicidad se deriva de la correspondencia con el SVD, que es igualmente único.


# Formas Canonicas o Normal

Una red está en forma canónica, si todas las matrices de enlaces son simultáneamente centros de ortogonalidad, fijando de forma unica la calibracion en cada uno de los enlaces de la red.

### Procedimiento para transformar una red a forma canónica:

*    Coloque una matriz de enlace $\sigma$, inicialmente establecida como matriz de identidad, en cada índice interno.

*    Use el enfoque de ortogonalización directa para establecer independientemente cada matriz de enlace $\sigma$ como un centro de ortogonalidad.

*    Realice una descomposicion SVD a cada enlace para que sea una matriz diagonal con elementos positivos ordenados descendientemente.

<img src="https://static.wixstatic.com/media/d91e93_89ebeadd221a42e0860ca130d32e422f~mv2.png/v1/fill/w_956,h_563,al_c,q_90/fig7.webp" width="500">

### Importancia de la forma canonica

Al configurar una red en forma canónica, uno puede:

*    truncar de manera óptima cualquier enlace a una dimensión reducida y comprender fácilmente el error de truncamiento correspondiente.
*    eliminar la ambigüedad de la calibracion en la descripción de la red, debido a la singularidad de la forma canónica.
*    extraer ciertos tipos de información de la red (como los valores esperados) de manera simplificada, debido a las cancelaciones que surgen en redes que contienen tensores isométricos.